In [ ]:
%%capture
%pip install -U sentence-transformers umap-learn

This notebook was run in Google Colab and used to explore different embeddings and dimensionality reduction strategies

In [1]:
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
from umap import UMAP

import pandas as pd
import numpy as np

In [13]:
df = pd.read_pickle("drive/MyDrive/Thesis-CDTM-Backup/data/predictions/df_chunks_embedding.pkl")

In [49]:
df = pd.read_pickle("drive/MyDrive/Thesis-CDTM-Backup/data/predictions/df_chunks_emb.pkl")
df = df.rename(columns={"chunks": "text"}).drop(columns="embeddings")

In [41]:
model = SentenceTransformer('BAAI/bge-large-en', device='cuda')
instruction = "Represent this sentence for searching relevant passages: "
queries = [instruction + t for t in df.text.tolist()]
embeddings = model.encode(queries, normalize_embeddings=True, show_progress_bar=True)
df["emb_bge_large_en"] = embeddings.tolist()
pca_model = PCA()
pca_model.fit(embeddings)
index_80 = np.min(np.argwhere(np.cumsum(pca_model.explained_variance_ratio_)> 0.8))
print(index_80, "Principal componenets needed to explain 80% of variance")
pca_model = PCA(n_components=index_80)
pca_embeddings = pca_model.fit_transform(embeddings)
df["emb_bge_large_en_pca_80"] = pca_embeddings.tolist()
pca_model = PCA(n_components=50)
pca_embeddings = pca_model.fit_transform(embeddings)
df["emb_bge_large_en_pca_50"] = pca_embeddings.tolist()
pca_model = PCA(n_components=10)
pca_embeddings = pca_model.fit_transform(embeddings)
df["emb_bge_large_en_pca_10"] = pca_embeddings.tolist()
for n_neighbors in [5, 10, 15]:
  for n_components in [5, 15, 30, 50]:
    umap_model = UMAP(n_neighbors=n_neighbors, n_components=n_components, min_dist=0.0, metric='cosine')
    embeddings_umap = umap_model.fit_transform(embeddings)
    df[f"emb_bge_large_en_umap_{n_neighbors}_{n_components}"] = embeddings_umap.tolist()

In [20]:
df.to_pickle("drive/MyDrive/Thesis-CDTM-Backup/data/predictions/df_chunks_embedding.pkl")

In [58]:
del model

In [ ]:
model = SentenceTransformer("intfloat/e5-large-v2", device='cuda')
instruction = "query: "
queries = [instruction + t for t in df.text.tolist()]
embeddings = model.encode(queries, normalize_embeddings=True, show_progress_bar=True)
df["emb_e5_large_v2"] = embeddings.tolist()
pca_model = PCA()
pca_model.fit(embeddings)
index_80 = np.min(np.argwhere(np.cumsum(pca_model.explained_variance_ratio_)> 0.8))
print(index_80, "Principal componenets needed to explain 80% of variance")
pca_model = PCA(n_components=index_80)
pca_embeddings = pca_model.fit_transform(embeddings)
df["emb_e5_large_v2_pca_80"] = pca_embeddings.tolist()
pca_model = PCA(n_components=50)
pca_embeddings = pca_model.fit_transform(embeddings)
df["emb_e5_large_v2_pca_50"] = pca_embeddings.tolist()
pca_model = PCA(n_components=10)
pca_embeddings = pca_model.fit_transform(embeddings)
df["emb_e5_large_v2_pca_10"] = pca_embeddings.tolist()
for n_neighbors in [5, 10, 15]:
  for n_components in [5, 15, 30, 50]:
    umap_model = UMAP(n_neighbors=n_neighbors, n_components=n_components, min_dist=0.0, metric='cosine')
    embeddings_umap = umap_model.fit_transform(embeddings)
    df[f"emb_e5_large_v2_umap_{n_neighbors}_{n_components}"] = embeddings_umap.tolist()

In [103]:
df.to_pickle("drive/MyDrive/Thesis-CDTM-Backup/data/predictions/df_chunks_embedding.pkl")